<h2>import Package </h2>

In [7]:
import tensorflow as tf
import numpy as np
import pandas as pd
import PIL 
import os
import shutil
import sys
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm.notebook import tqdm as tq
import ipywidgets as widgets
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import layers

<h2>get file in drive</h2>

In [16]:
from google.colab import drive
drive.mount('/content/drive')

# get data image
file = '/content/drive/My Drive/rockpaperscissors'

file_name = []
tag = []
full_path = []
for path, subdirs, files in os.walk(file):
    for name in files:
        full_path.append(os.path.join(path, name)) 
        tag.append(path.split('/')[-1])        
        file_name.append(name)
        
        
df = pd.DataFrame({"path":full_path,'file_name':file_name,"tag":tag})
df.groupby(['tag']).size()
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,path,file_name,tag
0,/content/drive/My Drive/rockpaperscissors/scis...,jMVjrTQfvJ0xLTOt.png,scissors
1,/content/drive/My Drive/rockpaperscissors/scis...,JlHdrLmHFDNY7Cy2.png,scissors
2,/content/drive/My Drive/rockpaperscissors/scis...,jL3CMVgsLdWznnKe.png,scissors
3,/content/drive/My Drive/rockpaperscissors/scis...,itSX8Pe5BwSSbcEb.png,scissors
4,/content/drive/My Drive/rockpaperscissors/scis...,IEE3kJIfbClsf54v.png,scissors
...,...,...,...
2183,/content/drive/My Drive/rockpaperscissors/rock...,Jgjf4Q4LunuuLJKq.png,rock
2184,/content/drive/My Drive/rockpaperscissors/rock...,izGsLJJdOIpr54aV.png,rock
2185,/content/drive/My Drive/rockpaperscissors/rock...,I9Ej8xGU5ivG8R2F.png,rock
2186,/content/drive/My Drive/rockpaperscissors/rock...,i1GxDtZ5awTW7Rsf.png,rock


<h2> Split data for train data and validation data </h2>

In [18]:
# split data
from sklearn.model_selection import train_test_split
X= df['path']
y= df['tag']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=300)

X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=100)

<h2> grouping data in train data, test data , and validation data </h2>

In [19]:
# grouping data 
df_tr = pd.DataFrame({'path':X_train
              ,'tag':y_train
             ,'set':'train'})

df_te = pd.DataFrame({'path':X_test
              ,'tag':y_test
             ,'set':'test'})

df_val = pd.DataFrame({'path':X_val
              ,'tag':y_val
             ,'set':'validation'})

<h2> cek data yang sudah di grouping </h2>

In [20]:
# cek data yang sudah di grouping
df_all = df_tr.append([df_te,df_val]).reset_index(drop=1)
print(df_all.groupby(['set','tag']).size(),'\n')

set         tag     
test        paper        70
            rock         72
            scissors     77
train       paper       561
            rock        591
            scissors    598
validation  paper        81
            rock         63
            scissors     75
dtype: int64 



<h2> buat paramaeter untuk networknya dan img data generator </h2>

In [22]:
# Parameter input untuk network
dim = (150, 150)
channel = (3, )
input_shape = dim + channel
# Batch size
batch_size = 16
# Epoch
epoch = 10

datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

<h2> membuat data generator 

In [23]:
train_generator = datagen.flow_from_dataframe(
dataframe=df_tr,
x_col="path",
y_col="tag",
target_size=dim,
class_mode='categorical',
shuffle=True)

val_generator = datagen.flow_from_dataframe(
dataframe=df_val,
x_col="path",
y_col="tag",
target_size=dim,
class_mode='categorical',
shuffle=True)

test_generator = datagen.flow_from_dataframe(
dataframe = df_te,
x_col="path",
y_col="tag",
target_size=dim,
class_mode='categorical',
shuffle=True)

num_class = len(test_generator.class_indices)

Found 1750 validated image filenames belonging to 3 classes.
Found 219 validated image filenames belonging to 3 classes.
Found 219 validated image filenames belonging to 3 classes.


In [24]:
def tf_data_generator(generator, input_shape):
    num_class = len(generator.class_indices)
    tf_generator = tf.data.Dataset.from_generator(
        lambda: generator,
        output_types=(tf.float32, tf.float32),
        output_shapes=([None
                        , input_shape[0]
                        , input_shape[1]
                        , input_shape[2]]
                       ,[None, num_class])
    )
    return tf_generator
train_data = tf_data_generator(train_generator, input_shape)
test_data = tf_data_generator(test_generator, input_shape)
val_data = tf_data_generator(val_generator, input_shape)

<h2> membuat model menggunakan sequentiel

In [25]:
model = Sequential()
model.add(Conv2D(128, (3, 3), padding='same', input_shape=input_shape))

model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_class))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer ='adam',
             metrics=["categorical_accuracy"])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 128)     3584      
_________________________________________________________________
activation (Activation)      (None, 150, 150, 128)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 148, 148, 32)      36896     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 74, 74, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 175232)            0         
_________________________________________________________________
dense (Dense)                (None, 512)               8

In [27]:
EPOCH = 3
model.fit(x=train_data,
        steps_per_epoch=len(train_generator),
        epochs=EPOCH,
        validation_data=val_data,
        validation_steps=len(val_generator),
        shuffle=True,
        verbose = 1)

Epoch 1/3
55/55 [==============================] - 713s 13s/step - loss: 1.1849 - categorical_accuracy: 0.6006 - val_loss: 0.6756 - val_categorical_accuracy: 0.7397
Epoch 2/3
55/55 [==============================] - 385s 7s/step - loss: 0.6996 - categorical_accuracy: 0.7280 - val_loss: 0.6317 - val_categorical_accuracy: 0.7260
Epoch 3/3
55/55 [==============================] - 387s 7s/step - loss: 0.5618 - categorical_accuracy: 0.7817 - val_loss: 0.4159 - val_categorical_accuracy: 0.8493


<h2> mengecek akurasi pada model diatas

In [28]:
loss, acc = model.evaluate(test_data,steps=len(test_generator),verbose=0)
print('Accuracy on training data: {:.4f} \nLoss on training data: {:.4f}'.format(acc,loss),'\n')
 
loss, acc = model.evaluate(test_data,steps=len(test_generator),verbose=0)
print('Accuracy on test data: {:.4f} \nLoss on test data: {:.4f}'.format(acc,loss),'\n')

Accuracy on training data: 0.8493 
Loss on training data: 0.4198 

Accuracy on test data: 0.8630 
Loss on test data: 0.3914 



<h2> membuat fungsi prediksi

In [29]:
import requests
from io import BytesIO
from PIL import Image
import numpy as np
# Parameters
input_size = (150,150)
#define input shape
channel = (3,)
input_shape = input_size + channel
#define labels

num_class = len(test_generator.class_indices)
label = train_generator.class_indices.keys()
labels = ['paper', 'rock', 'scissors']

def preprocess(img,input_size):
    nimg = img.convert('RGB').resize(input_size, resample= 0)
    img_arr = (np.array(nimg))/255
    return img_arr
def reshape(imgs_arr):
    return np.stack(imgs_arr, axis=0)

In [48]:
# read image
from google.colab import files
from io import BytesIO
uploaded = files.upload()
im = Image.open(BytesIO(uploaded['kung.png']))
X = preprocess(im,input_size)
X = reshape([X])
y = model.predict(X)
print( labels[np.argmax(y)], np.max(y) )

Saving kung.png to kung (9).png
rock 0.7404343
